In [1]:
import pandas as pd
pd.set_option('display.max_columns', 300)

In [15]:
# import packages for data cleaning and processing  
import pandas as pd
import numpy as np
from datetime import datetime

from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelBinarizer
from sklearn.preprocessing import LabelEncoder


import statsmodels.api as sm
import statsmodels.formula.api as smf
from statsmodels.formula.api import ols
import scipy.stats as scs

# import visualization modules
import seaborn as sns
import matplotlib.pyplot as plt

# set display options for Pandas dataframes
pd.set_option('display.max_rows', 200)
pd.set_option('display.max_columns', 200)

from sklearn.preprocessing import PolynomialFeatures

In [25]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
from sklearn import metrics
from sklearn import linear_model
from sklearn.feature_selection import SelectKBest, f_regression,mutual_info_regression
from sklearn.feature_selection import RFECV
import pickle

## Step 1: Read in hold out data, scalers, and best model

In [18]:
df = pd.read_csv('kc_house_data_test_features.csv', index_col=0)

In [19]:
df.head()

,id,date,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
0,1974300020,20140827T000000,4,2.50,2270,11500,1.0,0,0,3,8,1540,730,1967,0,98034,47.7089,-122.241,2020,10918
1,1974300020,20150218T000000,4,2.50,2270,11500,1.0,0,0,3,8,1540,730,1967,0,98034,47.7089,-122.241,2020,10918
2,3630020380,20141107T000000,3,2.50,1470,1779,2.0,0,0,3,8,1160,310,2005,0,98029,47.5472,-121.998,1470,1576
3,1771000290,20141203T000000,3,1.75,1280,16200,1.0,0,0,3,8,1030,250,1976,0,98077,47.7427,-122.071,1160,10565
4,5126310470,20150115T000000,4,2.75,2830,8126,2.0,0,0,3,8,2830,0,2005,0,98059,47.4863,-122.140,2830,7916


In [20]:
df = df.drop_duplicates(subset='id')
df['sale_date'] = [x[:8] for x in df.date]
df.sale_date = df.sale_date.apply(lambda x: datetime.strptime(x, '%Y%m%d'))
df.drop(columns='date', inplace=True)
df.drop(['id'], inplace=True, axis=1)

In [21]:
df.replace({'bedrooms': {33: 3}}, inplace=True)
df.replace({'bedrooms': {11: 1}}, inplace=True)
df['sale_age'] = df.sale_date.dt.year - df[['yr_built', 'yr_renovated']].max(axis=1)
df.replace({'sale_age': {-1: 0}}, inplace=True)
df['renovated'] = df.yr_renovated.apply(lambda x: x if x==0 else 1)
df['basement'] = df.sqft_basement.apply(lambda x: x if x==0 else 1)
df['viewed'] = df.view.apply(lambda x: x if x==0 else 1)
df.drop(['yr_built', 'yr_renovated', 'sale_date', 'sqft_basement', 'view'], inplace=True, axis=1)

In [22]:
df.bedrooms = df.bedrooms.astype('category')
df.bathrooms = df.bathrooms.astype('category')
df.floors = df.floors.astype('category')
df.condition = df.condition.astype('category')
df.grade = df.grade.astype('category')
df.renovated = df.renovated.astype('category')
df.basement = df.basement.astype('category')
df.viewed = df.viewed.astype('category')


In [23]:
df.head()

,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,condition,grade,sqft_above,zipcode,lat,long,sqft_living15,sqft_lot15,sale_age,renovated,basement,viewed
0,4,2.50,2270,11500,1.0,0,3,8,1540,98034,47.7089,-122.241,2020,10918,47,0,1,0
2,3,2.50,1470,1779,2.0,0,3,8,1160,98029,47.5472,-121.998,1470,1576,9,0,1,0
3,3,1.75,1280,16200,1.0,0,3,8,1030,98077,47.7427,-122.071,1160,10565,38,0,1,0
4,4,2.75,2830,8126,2.0,0,3,8,2830,98059,47.4863,-122.140,2830,7916,10,0,0,0
5,4,2.25,1970,7125,1.5,0,3,7,1500,98115,47.6725,-122.293,1980,4750,106,0,1,0


In [24]:
index_dum = df[['bedrooms', 'bathrooms', 'floors', 'condition', 'grade']].columns
df_dum = pd.get_dummies(data=df, columns=index_dum, drop_first=True, prefix=['bdr', 'bth', 'flr', 'cnd', 'grd'])
# df_features = df_dum.drop('price', axis=1)
# target = df['price']
poly = PolynomialFeatures(degree=2, include_bias=False)
poly_data = poly.fit_transform(df_dum)
poly_columns = poly.get_feature_names(df_dum.columns)
df_poly = pd.DataFrame(poly_data, columns=poly_columns)
df_poly

,sqft_living,sqft_lot,waterfront,sqft_above,zipcode,lat,long,sqft_living15,sqft_lot15,sale_age,renovated,basement,viewed,bdr_1,bdr_2,bdr_3,bdr_4,bdr_5,bdr_6,bdr_7,bdr_8,bdr_9,bdr_10,bth_0.75,bth_1.0,bth_1.25,bth_1.5,bth_1.75,bth_2.0,bth_2.25,bth_2.5,bth_2.75,bth_3.0,bth_3.25,bth_3.5,bth_3.75,bth_4.0,bth_4.25,bth_4.5,bth_4.75,bth_5.0,bth_5.25,bth_5.5,bth_6.0,bth_6.25,bth_6.5,flr_1.5,flr_2.0,flr_2.5,flr_3.0,flr_3.5,cnd_2,cnd_3,cnd_4,cnd_5,grd_4,grd_5,grd_6,grd_7,grd_8,grd_9,grd_10,grd_11,grd_12,grd_13,sqft_living^2,sqft_living sqft_lot,sqft_living waterfront,sqft_living sqft_above,sqft_living zipcode,sqft_living lat,sqft_living long,sqft_living sqft_living15,sqft_living sqft_lot15,sqft_living sale_age,sqft_living renovated,sqft_living basement,sqft_living viewed,sqft_living bdr_1,sqft_living bdr_2,sqft_living bdr_3,sqft_living bdr_4,sqft_living bdr_5,sqft_living bdr_6,sqft_living bdr_7,sqft_living bdr_8,sqft_living bdr_9,sqft_living bdr_10,sqft_living bth_0.75,sqft_living bth_1.0,sqft_living bth_1.25,sqft_living bth_1.5,sqft_living bth_1.75,sqft_living bth_2.0,sqft_living bth_2.25,sqft_living bth_2.5,sqft_living bth_2.75,sqft_living bth_3.0,sqft_living bth_3.25,sqft_living bth_3.5,...,cnd_2 grd_5,cnd_2 grd_6,cnd_2 grd_7,cnd_2 grd_8,cnd_2 grd_9,cnd_2 grd_10,cnd_2 grd_11,cnd_2 grd_12,cnd_2 grd_13,cnd_3^2,cnd_3 cnd_4,cnd_3 cnd_5,cnd_3 grd_4,cnd_3 grd_5,cnd_3 grd_6,cnd_3 grd_7,cnd_3 grd_8,cnd_3 grd_9,cnd_3 grd_10,cnd_3 grd_11,cnd_3 grd_12,cnd_3 grd_13,cnd_4^2,cnd_4 cnd_5,cnd_4 grd_4,cnd_4 grd_5,cnd_4 grd_6,cnd_4 grd_7,cnd_4 grd_8,cnd_4 grd_9,cnd_4 grd_10,cnd_4 grd_11,cnd_4 grd_12,cnd_4 grd_13,cnd_5^2,cnd_5 grd_4,cnd_5 grd_5,cnd_5 grd_6,cnd_5 grd_7,cnd_5 grd_8,cnd_5 grd_9,cnd_5 grd_10,cnd_5 grd_11,cnd_5 grd_12,cnd_5 grd_13,grd_4^2,grd_4 grd_5,grd_4 grd_6,grd_4 grd_7,grd_4 grd_8,grd_4 grd_9,grd_4 grd_10,grd_4 grd_11,grd_4 grd_12,grd_4 grd_13,grd_5^2,grd_5 grd_6,grd_5 grd_7,grd_5 grd_8,grd_5 grd_9,grd_5 grd_10,grd_5 grd_11,grd_5 grd_12,grd_5 grd_13,grd_6^2,grd_6 grd_7,grd_6 grd_8,grd_6 grd_9,grd_6 grd_10,grd_6 grd_11,grd_6 grd_12,grd_6 grd_13,grd_7^2,grd_7 grd_8,grd_7 grd_9,grd_7 grd_10,grd_7 grd_11,grd_7 grd_12,grd_7 grd_13,grd_8^2,grd_8 grd_9,grd_8 grd_10,grd_8 grd_11,grd_8 grd_12,grd_8 grd_13,grd_9^2,grd_9 grd_10,grd_9 grd_11,grd_9 grd_12,grd_9 grd_13,grd_10^2,grd_10 grd_11,grd_10 grd_12,grd_10 grd_13,grd_11^2,grd_11 grd_12,grd_11 grd_13,grd_12^2,grd_12 grd_13,grd_13^2
0,2270.0,11500.0,0.0,1540.0,98034.0,47.7089,-122.241,2020.0,10918.0,47.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,5152900.0,26105000.0,0.0,3495800.0,222537180.0,108299.203,-277487.07,4585400.0,24783860.0,106690.0,0.0,2270.0,0.0,0.0,0.0,0.0,2270.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2270.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1470.0,1779.0,0.0,1160.0,98029.0,47.5472,-121.998,1470.0,1576.0,9.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,2160900.0,2615130.0,0.0,1705200.0,144102630.0,69894.384,-179337.06,2160900.0,2316720.0,13230.0,0.0,1470.0,0.0,0.0,0.0,1470.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1470.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0

In [26]:
final = df_poly[['sqft_living', 'sqft_above', 'sqft_living15', 'sqft_living^2',
       'sqft_living sqft_above', 'sqft_living zipcode', 'sqft_living lat',
       'sqft_living long', 'sqft_living sqft_living15', 'sqft_living viewed',
       'sqft_above^2', 'sqft_above zipcode', 'sqft_above lat',
       'sqft_above long', 'sqft_above sqft_living15', 'sqft_above viewed',
       'zipcode sqft_living15', 'lat sqft_living15', 'long sqft_living15',
       'sqft_living15^2']]

In [ ]:
# final_scaler = read_pickle(filename)
# final_model = read_pickle(filename)

## Step 2: Feature Engineering for holdout set

Remember we have to perform the same transformations on our holdout data (feature engineering, extreme values, and scaling) that we performed on the original data.

In [ ]:
# transformed_holdout = final_scaler(holdout)

## Step 3: Predict the holdout set

In [ ]:
# final_answers = final_model.predict(transformed_holdout)

## Step 4: Export your predictions

In [ ]:
# final_answer.to_csv('housing_preds_your_name.csv')